In [3]:
# Importando bibliotecas
# Vamos importar as libs que pretendemos usar ao longo do projeto
import pandas as pd
import math
import matplotlib.pyplot as plt

%matplotlib inline

# Lendo arquivo de dados de vulcões
df = pd.read_csv('Volcano-Eruptions.csv')

# Testando se a importação ocorreu bem
df.head()

,Volcano Number,Volcano Name,Country,Primary Volcano Type,Activity Evidence,Last Known Eruption,Region,Subregion,Latitude,Longitude,Elevation (m),Dominant Rock Type,Tectonic Setting
0,210010,West Eifel Volcanic Field,Germany,Maar(s),Eruption Dated,8300 BCE,Mediterranean and Western Asia,Western Europe,50.170,6.85,600,Foidite,Rift zone / Continental crust (>25 km)
1,210020,Chaine des Puys,France,Lava dome(s),Eruption Dated,4040 BCE,Mediterranean and Western Asia,Western Europe,45.775,2.97,1464,Basalt / Picro-Basalt,Rift zone / Continental crust (>25 km)
2,210030,Olot Volcanic Field,Spain,Pyroclastic cone(s),Evidence Credible,Unknown,Mediterranean and Western Asia,Western Europe,42.170,2.53,893,Trachybasalt / Tephrite Basanite,Intraplate / Continental crust (>25 km)
3,210040,Calatrava Volcanic Field,Spain,Pyroclastic cone(s),Eruption Dated,3600 BCE,Mediterranean and Western Asia,Western Europe,38.870,-4.02,1117,Basalt / Picro-Basalt,Intraplate / Continental crust (>25 km)
4,211001,Larderello,Italy,Explosion crater(s),Eruption Observed,1282 CE,Mediterranean and Western Asia,Italy,43.250,10.87,500,No Data (checked),Subduction zone / Continental crust (>25 km)


## Limpeza de dados

Nesta etapa verificar quais colunas apresentam dados faltantes e decidir o que fazer fazer com elas

Usaremos agora um método que identifica quais colunas possuem dados vazios, e quantos são:

In [4]:
df.isnull().sum()

Volcano Number           0
Volcano Name             0
Country                  0
Primary Volcano Type     0
Activity Evidence        0
Last Known Eruption      0
Region                   0
Subregion                0
Latitude                 0
Longitude                0
Elevation (m)            0
Dominant Rock Type      14
Tectonic Setting         5
dtype: int64

## Os dados faltantes

As colunas com dados vazios são:

- Dominant Rock Type
- Tectonic Setting

Vamos verificar o tipo dos dados, pois com essa informação poderemos tomar melhor nossa decisão

In [9]:
rocktype     = df['Dominant Rock Type'].dtype
tectonictype = df['Tectonic Setting'].dtype

print("O tipo da coluna 'Dominant Rock Type' é {} e o tipo de Tectonic Setting é {}".format(rocktype,tectonictype))

O tipo da coluna 'Dominant Rock Type' é object e o tipo de Tectonic Setting é object


## Definindo o que faremos:

Aqui cabe uma importante observação, colunas com o tipo 'object' no pandas são equivalentes a strings, ou seja, texto

Assim sendo strings são dados importantes, meu julgamento é que deveriamos remover estas linhas faltantes para não comprometer a integridade dos dados

Acho errado 'inventarmos' um valor para preencher estas linhas apenas para usar o dataset completo

Porém caso fosse uma coluna numérica, poderiamos até tirar a média e completar os valores faltantes com ela, mas neste caso específico minha decisão é de remover estas colunas